In [ ]:
from lxml import html
import requests
import numpy as np
import pandas as pd
import pickle

In [ ]:
nominations = {'amenity': ['cafe', 'fast_food', 'food_court', 'pub', 'restaurant', 'kindergarten',\
                       'school', 'university', 'bank', 'clinic', 'dentist', 'doctors', 'hospital', 'arts_centre',\
                       'cinema', 'nightclub', 'theatre', 'marketplace', 'public_bath', 'place_of_worship'],\
           'highway': ['bus_stop'],\
           'railway': ['subway_entrance', 'tram_stop'],\
           'shop':  ['department_store', 'mall', 'supermarket', 'convenience', 'general'],\
           'building': ['apartments'],\
           'man_made':['works'],\
           'leisure':['beach_resort', 'dance', 'fitness_centre', 'ice_rink', 'park', 'pitch', 'playground',\
                      'sports_centre', 'stadium', 'swimming_pool']}

In [ ]:
objects = {}
for i in nominations.keys():
    for j in nominations[i]:
        objects[i + '_' + j] = []
print objects

In [ ]:
min_latitude = 55.5737
max_latitude = 55.9084
min_longitude = 37.3618
max_longitude = 37.8438

In [ ]:
lat_div = 5
lon_div = 5

In [ ]:
delta_lat, delta_lon = (max_latitude - min_latitude) / lat_div, (max_longitude - min_longitude) / lon_div
current_lat, current_lon = min_latitude, min_longitude

In [ ]:
house_addresses = []

In [ ]:
def get_string(x):
    if x:
        return unicode(x[0])
    else:
        return ''

In [ ]:
for i in range(lat_div):
    for j in range(lon_div):
        right_top_point_lat = current_lat + delta_lat
        right_top_point_lon = current_lon + delta_lon
        !osmosis --read-pbf file=central-fed-district-latest.osm.pbf --bounding-box bottom=$current_lat top=$right_top_point_lat left=$current_lon right=$right_top_point_lon --write-xml file=moscow.osm
        
        !./osmconvert moscow.osm --all-to-nodes -o=temp.osm
        !./osmfilter temp.osm --keep="amenity= or building=apartments or highway=bus_stop or shop= or railway=subway_entrance or railway=tram_stop or man_made=works or leisure=" --drop-author --drop-version -o=amenities.osm
        
        for nomin_category in nominations.keys():
            for nomin in nominations[nomin_category]:
                f = open('amenities.osm', 'r')
                tree = html.fromstring(f.read())
                path = "//node[./tag/@k='%s' and ./tag/@v='%s']" % (nomin_category, nomin)
                for node in tree.xpath(path):
                    objects[nomin_category + '_' + nomin].append([node.get('lat'), node.get('lon')])
                    if nomin_category == 'building':
                        hn1 = node.xpath("./tag[@k='addr:housenumber']/@v")
                        hn2 = node.xpath("./tag[@k='addr:housenumber2']/@v")
                        st = node.xpath("./tag[@k='addr:street']/@v")
                        house_addresses.append([get_string(hn1), get_string(hn2), get_string(st)])
                    
        current_lon += delta_lon
        print 'progress is %d/25' % (i * lat_div + j + 1)
    current_lon = min_longitude
    current_lat += delta_lat

In [ ]:
for i in objects.keys():
    print i, len(objects[i])

In [ ]:
with open('objects.pickle', 'wb') as f:
    pickle.dump(objects, f)

In [ ]:
with open('house_addresses.pickle', 'wb') as f:
    pickle.dump(house_addresses, f)